In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

In [2]:


batch_size = 256
epochs = 10
embed_size = 100



In [3]:
import pandas as pd
test2=pd.read_csv('../input/hate-speech-detection/toxic_test.csv')
train2=pd.read_csv('../input/hate-speech-detection/toxic_train.csv')

In [4]:
del test2['Unnamed: 0']
del train2['Unnamed: 0']


In [5]:
train2['text']=train2['comment_text']
test2['text']=test2['comment_text']
del train2['comment_text']
del test2['comment_text']


In [6]:
test2.head()


,toxic,text
0,0,Thank you for understanding. I think very high...
1,0,:Dear god this site is horrible.
2,0,"""::: Somebody will invariably try to add Relig..."
3,0,""" \r\n\r\n It says it right there that it IS a..."
4,0,""" \r\n\r\n == Before adding a new product to t..."


In [7]:
train2.head()

,toxic,text
0,0,Explanation\r\nWhy the edits made under my use...
1,0,D'aww! He matches this background colour I'm s...
2,0,"Hey man, I'm really not trying to edit war. It..."
3,0,"""\r\nMore\r\nI can't make any real suggestions..."
4,0,"You, sir, are my hero. Any chance you remember..."


In [8]:
from nltk.corpus import stopwords
import re,string,unicodedata
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)


In [9]:
from bs4 import BeautifulSoup
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Removing URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Removing the stopwords from text
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
def lowering(text):
    text=text.lower() 
    return text

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    text =lowering(text)
    return text



In [10]:
test2['text']=test2['text'].apply(denoise_text)
test2

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:398: UserWarning: "http://en.wikipedia.org/wiki/Half-Life_%28series%29" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  markup
/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:398: UserWarning: "http://es.wikipedia.org/wiki/Usuario_Discusi%C3%B3n:Sarah_W." looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  markup
/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:398: UserWarning: "http://en.wikipedia.org/wiki/Rayleigh%E2%80%93B%C3%A9nard_convection" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  markup


,toxic,text
0,0,thank understanding. think highly would revert...
1,0,:dear god site horrible.
2,0,"""::: somebody invariably try add religion? rea..."
3,0,"says right type. """"type"""" institution needed c..."
4,0,"== adding new product list, make sure relevant..."
...,...,...
63973,0,":jerome, see never got around this…! i’m surpr..."
63974,0,==lucky bastard== heh famous now. kida envy th...
63975,0,==shame all!!!== want speak gays romanians...
63976,1,mel gibson nazi bitch makes shitty movies. muc...


In [11]:
train2['text']=train2['text'].apply(denoise_text)
train2

/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:398: UserWarning: "http://en.wikipedia.org/wiki/Wikipedia_talk:No_original_research/archive15#YouTube_art_as_primary_source" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  markup
/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:398: UserWarning: "http://finance.yahoo.com/news/7-fascinating-nuggets-another-bewildering-150348488.html" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  markup
/opt/conda/lib/python3.7/site-packages/bs4/__init__.py:398: UserWarning: "http://en.wikipedia.org/wiki/Wikipedia:ELYES" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, a

,toxic,text
0,0,explanation edits made username hardcore metal...
1,0,d'aww! matches background colour i'm seemingly...
2,0,"hey man, i'm really trying edit war. guy const..."
3,0,can't make real suggestions improvement wonder...
4,0,"you, sir, hero. chance remember page that's on?"
...,...,...
159566,0,""":::::and second time asking, view completely ..."
159567,0,ashamed horrible thing put talk page. 128.61.1...
159568,0,"spitzer umm, theres actual article prostitutio..."
159569,0,looks like actually put speedy first version d...


In [12]:
x_train=train2['text']
x_test=test2['text']
y_train=train2['toxic']
y_test=test2['toxic']

In [13]:
max_features = 10000
maxlen = 300

In [14]:

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = sequence.pad_sequences(tokenized_train, maxlen=maxlen)


tokenized_test = tokenizer.texts_to_sequences(x_test)
X_test = sequence.pad_sequences(tokenized_test, maxlen=maxlen)


In [15]:
EMBEDDING_FILE = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.100d.txt'

In [16]:
def get_coefs(word, *arr): 
    return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

In [17]:
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if (await self.run_code(code, result,  async_=asy)):


In [18]:


epochs = 10
embed_size = 100
filters= 250
kernel_size=3
hidden_dims= 250



In [19]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns

In [20]:

from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import seaborn as sns
model2 = Sequential()
model2.add(Embedding(max_features, output_dim=embed_size, weights=[embedding_matrix], input_length=maxlen, trainable=False))
model2.add(Dropout(0.2))
model2.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu'))
model2.add(MaxPooling1D())
model2.add(Conv1D(filters,
                 5,
                 padding='valid',
                 activation='relu'))
model2.add(GlobalMaxPooling1D())
model2.add(Dense(hidden_dims, activation='relu'))
model2.add(Dropout(0.2))

model2.add(Dense(1, activation='sigmoid'))
model2.summary()
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          1000000   
_________________________________________________________________
dropout (Dropout)            (None, 300, 100)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 298, 250)          75250     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 149, 250)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 145, 250)          312750    
_________________________________________________________________
global_max_pooling1d (Global (None, 250)               0         
_________________________________________________________________
dense (Dense)                (None, 250)               6

In [ ]:
hist = model2.fit(x_train,y_train,
                    batch_size= 32,
                    epochs=60,
                    validation_data= (X_test,y_test)
                    )

1160/4987 [=====>........................] - ETA: 26s - loss: 0.1705 - accuracy: 0.9416

In [ ]:
y_pred = model2.predict(X_test,verbose=1,batch_size=32)
y_pred = [0 if y[0] < 0.5 else 1 for y in y_pred]
print("Accuracy: {:.2f}%".format(accuracy_score(y_test, y_pred) * 100))
print("F1 Score: {:.6f}".format(f1_score(y_test, y_pred, average='macro')))



In [ ]:


# Plot loss vs epochs
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training loss vs. Epochs')
plt.legend()
plt.show()



In [ ]:
# Plot accuracy vs epochs
plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='validation')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training accuracy vs. Epochs')
plt.legend()
plt.show()
